In [39]:
import numpy as np 
import pandas as pd
import os
import gc
pd.set_option("max_colwidth", 40)

In [2]:
data_path = "../data/imdb-dataset/"

In [3]:
os.listdir(data_path)

['name.basics.tsv',
 'title.akas.tsv',
 'title.basics.tsv',
 'title.principals.tsv',
 'title.ratings.tsv']

In [4]:
name_basics = pd.read_table(data_path+"name.basics.tsv")

In [5]:
title_akas = pd.read_table(data_path+"title.akas.tsv")

C:\Users\tanmu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
title_basics = pd.read_table(data_path+"title.basics.tsv")

C:\Users\tanmu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
title_principals = pd.read_table(data_path+"title.principals.tsv")

In [8]:
title_ratings = pd.read_table(data_path+"title.ratings.tsv")

In [9]:
# convert unknown startYear data to 0 
title_basics.loc[title_basics['startYear']=='\\N','startYear'] = 0
# convert to numeric
title_basics.startYear = title_basics.startYear.astype(np.int64)

In [10]:
df = pd.read_csv("../data/movie_data.tsv", index_col=0)

In [11]:
tmdb = pd.read_csv("../data/tmdb-dataset/movie_tmdb.csv", encoding='utf8', engine='python')

In [37]:
df.tail()

,user_name,movie_name,release_year,user_review,user_rating,review_date,rewatched,review_likes
293465,/silentjoe13/,Abduction,2011,I had a feeling this film ...,1,"29 Sep, 2011",0,1
293466,/silentjoe13/,GoodFellas,1990,Best gangster film ever. E...,10,"01 Mar, 2012",0,4
293467,/silentjoe13/,Drive,2011,Probably the best surprise...,10,"25 Sep, 2011",0,6
293468,/silentjoe13/,Cinema Paradiso,1988,Not only is this one of th...,10,"01 Mar, 2012",0,6
293469,/silentjoe13/,Manhattan,1979,"Witty, whimsical, poignant...",10,"29 Feb, 2012",0,5


In [14]:
tmdb.loc[~(tmdb.release_date.isnull()),'year'] = tmdb.release_date.apply(lambda x: str(x)[:4])

In [15]:
pd.options.display.max_columns = 50

In [16]:
tmdb['year'] = tmdb.year.fillna(0).astype(np.int64)

In [17]:
tmdb.isnull().any(axis=0)

adult                    False
backdrop_path             True
belongs_to_collection     True
budget                    True
genres                    True
homepage                  True
id                        True
imdb_id                   True
original_language         True
original_title            True
overview                  True
popularity                True
poster_path               True
production_companies      True
production_countries      True
release_date              True
revenue                   True
runtime                   True
spoken_languages          True
status                    True
tagline                   True
title                     True
video                     True
vote_average              True
vote_count                True
year                     False
dtype: bool

In [18]:
name_basics.head(1)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0043044"


In [19]:
title_akas.head(1)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0


In [20]:
title_basics.head(1)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"


In [21]:
title_principals.head(1)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"


In [22]:
title_ratings.head(1)

,tconst,averageRating,numVotes
0,tt0000001,5.6,1550


In [26]:
tmdb.columns

Index(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'year'],
      dtype='object')

In [28]:
df_v1 = pd.merge(df, title_basics[title_basics.titleType=='movie'],  how='left',
                  left_on=['movie_name','release_year'], 
                  right_on = ['primaryTitle','startYear'])

In [29]:
sum(df_v1.primaryTitle.isnull())

55520

In [31]:
df_v1.shape

(299379, 17)

In [32]:
df.shape

(293470, 8)

In [33]:
print(f'{df_v1.shape[0] - df.shape[0]} entries added due to duplicate matches')

5909 entries added due to duplicate matches


In [34]:
df_v2 = pd.merge(df_v1, tmdb,  how='left',
                     left_on=['movie_name','release_year','tconst'], 
                     right_on = ['title','year','imdb_id'],
                     suffixes=("_orig","_tmdb"))

In [40]:
df_v2.head(1)

,user_name,movie_name,release_year,user_review,user_rating,review_date,rewatched,review_likes,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_orig,adult,backdrop_path,belongs_to_collection,budget,genres_tmdb,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,/worsethan/,Star Wars: The Rise of Skywalker,2019,"""I remember when I was with Special ...",10,"21 Dec, 2019",0,187,tt2527338,movie,Star Wars: The Rise of Skywalker,Star Wars: Episode IX - The Rise of ...,0.0,2019.0,\N,155,"Action,Adventure,Fantasy",False,/jOzrELAzFxtMx2I4uDGHOotdfsS.jpg,"{'id': 10, 'name': 'Star Wars Collec...",0,"[{'id': 28, 'name': 'Action'}, {'id'...",https://www.starwars.com/films/star-...,181812.0,tt2527338,en,Star Wars: The Rise of Skywalker,The next installment in the franchis...,34.725,/db32LaOibwEliAmSL2jjDF6oDdj.jpg,"[{'id': 1, 'logo_path': '/o86Dbpburj...","[{'iso_3166_1': 'JO', 'name': 'Jorda...",2019-12-18,0.0,155.0,"[{'iso_639_1': 'en', 'name': 'Englis...",Post Production,Every generation has a legend,Star Wars: The Rise of Skywalker,False,0.0,0.0,2019.0


In [41]:
df_v2.shape

(299390, 43)

In [46]:
print(f'{df_v2.shape[0] - df.shape[0]} entries added due to duplicate matches after combining with tmdb dataset from Kaggle\n\
[https://www.kaggle.com/mathlasker/tmdballmovies]')

5920 entries added due to duplicate matches after combining with tmdb dataset from Kaggle
[https://www.kaggle.com/mathlasker/tmdballmovies]


In [51]:
print(f'Total titles missing from tmdb dataset: {df_v2.title.isnull().sum()} \n\
Total titles missing from tmdb dataset imdb dataset: {df_v1.primaryTitle.isnull().sum()} \n\
Total titles missing after merging both: {df_v2.title.fillna(df_v1.primaryTitle).isnull().sum()}')

Total titles missing from tmdb dataset: 67637 
Total titles missing from tmdb dataset imdb dataset: 55520 
Total titles missing after merging both: 24787


In [56]:
df_v2["title_inferred"] = df_v2["title"].fillna(df_v2["primaryTitle"])

In [59]:
df_v2.loc[[11,24,25,29,74,88,89,97],['user_name','movie_name','primaryTitle','title','imdb_id','tconst','runtimeMinutes',
                                        'runtime','user_review']]

,user_name,movie_name,primaryTitle,title,imdb_id,tconst,runtimeMinutes,runtime,user_review
11,/worsethan/,Once Upon a Time… in Hollywood,NaN,NaN,NaN,NaN,NaN,NaN,Blasphemously watched this on a post...
24,/worsethan/,Hotel Mumbai,Hotel Mumbai,NaN,NaN,tt5461944,123,NaN,"Got that United 93 ""shit happens"" vibe."
25,/worsethan/,Return of the Living Dead Part II,NaN,NaN,NaN,NaN,NaN,NaN,Inferior to the original in every wa...
29,/worsethan/,68 Kill,68 Kill,NaN,NaN,tt5189894,93,NaN,Flagrant anti-thot propaganda.
74,/worsethan/,Molly’s Game,NaN,NaN,NaN,NaN,NaN,NaN,The most annoying thing about Sorkin...
88,/worsethan/,Vice,Vice,Vice,tt6266538,tt6266538,132,132.0,Like watching someone try to eat a H...
89,/worsethan/,Zama,Zama,Zama,tt3409848,tt3409848,115,115.0,Something in my brain rejects movies...
97,/worsethan/,Triangle,Triangle,Triangle,tt1187064,tt1187064,99,99.0,I'm realizing these movies just kind...


In [61]:
# Lets look at duplicate entries
duplicates = pd.concat(g for _, g in df_v2.groupby("user_review") if len(g) > 1)

In [62]:
# Peek into duplicates
duplicates[duplicates.user_review.str.strip()!=""].head()

,user_name,movie_name,release_year,user_review,user_rating,review_date,rewatched,review_likes,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_orig,adult,backdrop_path,belongs_to_collection,budget,genres_tmdb,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,title_inferred
56555,/spiderpig/,Alice in Wonderland,2010,Jesus Christ...what an absolu...,1,"09 Jan, 2020",0,18,tt1014759,movie,Alice in Wonderland,Alice in Wonderland,0.0,2010.0,\N,108,"Adventure,Family,Fantasy",False,/aBCjJWQkEsn3Y3nbx1IOKRLalqa.jpg,"{'id': 261307, 'name': 'Alice in Won...",200000000,"[{'id': 10751, 'name': 'Family'}, {'...",http://disney.go.com/wonderland/,12155.0,tt1014759,en,Alice in Wonderland,"Alice, an unpretentious and individu...",21.84,/o0kre9wRCZz3jjSjaru7QU0UtFz.jpg,"[{'id': 2, 'logo_path': '/wdrCwmRnLF...","[{'iso_3166_1': 'US', 'name': 'Unite...",2010-03-03,1.025467e+09,108.0,"[{'iso_639_1': 'en', 'name': 'Englis...",Released,You're invited to a very important date,Alice in Wonderland,False,6.6,9330.0,2010.0,Alice in Wonderland
56556,/spiderpig/,Alice in Wonderland,2010,Jesus Christ...what an absolu...,1,"09 Jan, 2020",0,18,tt2049386,movie,Alice in Wonderland,Alice in Wonderland,0.0,2010.0,\N,99,"Fantasy,Musical",False,NaN,NaN,0,[],NaN,423971.0,tt2049386,en,Alice in Wonderland,An adaptation of the 1886 musical “A...,0.888,/7zKOHqU1VFZB0FwhsrrBjtlTXJ0.jpg,[],[],2010-01-01,0.000000e+00,99.0,[],Released,NaN,Alice in Wonderland,False,6.0,1.0,2010.0,Alice in Wonderland
206247,/andredenervaux/,Love,2015,Theres nothing after death Ive hea...,9,"27 Feb, 2018",0,30,tt3774694,movie,Love,Love,0.0,2015.0,\N,135,"Drama,Romance",False,/aoZaxMnJZEfeHuzjllBFoTuPgGR.jpg,NaN,3000000,"[{'id': 18, 'name': 'Drama'}, {'id':...",NaN,292431.0,tt3774694,en,Love,Murphy is an American living in Pari...,4.258,/dw7Lbq7luiH11cbL5MlJwUdxkUk.jpg,"[{'id': 856, 'logo_path': '/3tfzS2Cr...","[{'iso_3166_1': 'BE', 'name': 'Belgi...",2015-07-06,2.490830e+05,134.0,"[{'iso_639_1': 'en', 'name': 'Englis...",Released,NaN,Love,False,6.0,903.0,2015.0,Love
206248,/andredenervaux/,Love,2015,Theres nothing after death Ive hea...,9,"27 Feb, 2018",0,30,tt4777250,movie,Love,Yêu,0.0,2015.0,\N,\N,Romance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Love
206249,/andredenervaux/,Love,2015,Theres nothing after death Ive hea...,9,"27 Feb, 2018",0,30,tt8212482,movie,Love,Love,0.0,2015.0,\N,\N,Fantasy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Love


Looks like most duplicates are from the same name productions from the same year.

In [64]:
print(f"The number of duplicates: {duplicates.shape[0]}")

The number of duplicates: 13815


In [71]:
titles_wo_people_info = set(df_v2.tconst.unique()).difference(set(title_principals.tconst.unique()))

In [113]:
def get_people(imdb_mov_id):
    if ((len(imdb_mov_id) < 7) | (imdb_mov_id in titles_wo_people_info)):
        return np.nan, np.nan, np.nan, np.nan, np.nan
    temp = title_principals.loc[title_principals['tconst']==imdb_mov_id]
    people_codes = temp.nconst
    row_logical = name_basics.nconst.isin(people_codes)
    names = " ".join([name.replace(" ","") for name in name_basics.loc[row_logical,"primaryName"]])
    birth_years = " ".join(name_basics.loc[row_logical,"birthYear"])
    death_years = " ".join(name_basics.loc[row_logical,"deathYear"])
    professions = " ".join(name_basics.loc[row_logical,"primaryProfession"])
    known_for_movies = " ".join(name.replace(","," ") for name in name_basics.loc[row_logical,"knownForTitles"])
    return names, birth_years, death_years, professions, known_for_movies

In [160]:
# Try people info generation with a small sample
abc = df_v2.head().copy()

In [162]:
import time

def generate_people_feats(df):
    for col in ["names", "birth_years", "death_years", "professions", "known_for_movies"]:
        df[col] = pd.core.series.Series(dtype='object')
    for mov_id in df['tconst'].unique():
        names, birth_years, death_years, professions, known_for_movies = get_people(mov_id)
        logical_idx = df.tconst==mov_id
        df.loc[logical_idx, 'names'] = names
        df.loc[logical_idx, 'birth_years'] = birth_years
        df.loc[logical_idx, 'death_years'] = death_years
        df.loc[logical_idx, 'professions"'] = professions
        df.loc[logical_idx, 'known_for'] = known_for_movies

In [163]:
t0 = time.time()
generate_people_feats(abc)
t1 = time.time()
print(t1-t0)

39.46292686462402


In [166]:
abc

,user_name,movie_name,release_year,user_review,user_rating,review_date,rewatched,review_likes,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_orig,adult,backdrop_path,belongs_to_collection,budget,genres_tmdb,homepage,id,imdb_id,...,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,title_inferred,names,birth_years,death_years,professions,known_for_movies,"professions""",known_for
0,/worsethan/,Star Wars: The Rise of Skywalker,2019,"""I remember when I was with Special ...",10,"21 Dec, 2019",0,187,tt2527338,movie,Star Wars: The Rise of Skywalker,Star Wars: Episode IX - The Rise of ...,0.0,2019.0,\N,155,"Action,Adventure,Fantasy",False,/jOzrELAzFxtMx2I4uDGHOotdfsS.jpg,"{'id': 10, 'name': 'Star Wars Collec...",0,"[{'id': 28, 'name': 'Action'}, {'id'...",https://www.starwars.com/films/star-...,181812.0,tt2527338,...,Star Wars: The Rise of Skywalker,The next installment in the franchis...,34.725,/db32LaOibwEliAmSL2jjDF6oDdj.jpg,"[{'id': 1, 'logo_path': '/o86Dbpburj...","[{'iso_3166_1': 'JO', 'name': 'Jorda...",2019-12-18,0.0,155.0,"[{'iso_639_1': 'en', 'name': 'Englis...",Post Production,Every generation has a legend,Star Wars: The Rise of Skywalker,False,0.0,0.0,2019.0,Star Wars: The Rise of Skywalker,GeorgeLucas KathleenKennedy ChrisTer...,1944 1953 1976 1966 1976 \N 1983 199...,\N \N \N \N \N \N \N \N \N \N,NaN,NaN,"writer,producer,director producer,mi...",tt0076759 tt0066434 tt0082971 tt0069...
1,/worsethan/,The Report,2019,I realize that I prefer the Adam McK...,7,"16 Dec, 2019",0,48,tt8236336,movie,The Report,The Report,0.0,2019.0,\N,119,"Drama,Thriller",False,/yRNrOJyyeHJjgtJNLcqpo7rQyS6.jpg,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,524348.0,tt8236336,...,The Report,"The story of Daniel Jones, lead inve...",7.59,/sG5NI6TMNR9ftOdTIXmG0hrBYSY.jpg,"[{'id': 30288, 'logo_path': None, 'n...",[],2019-09-12,0.0,118.0,"[{'iso_639_1': 'en', 'name': 'Englis...",Released,Truth matters,The Report,False,7.3,6.0,2019.0,The Report,AnnetteBening StevenSoderbergh Jenni...,1958 1963 \N 1971 1957 \N \N \N \N 1983,\N \N \N \N \N \N \N \N \N \N,NaN,NaN,"actress,soundtrack,executive produce...",tt4385888 tt0169547 tt0112346 tt0842...
2,/worsethan/,Uncut Gems,2019,All the greasy tri-state claustropho...,9,"18 Dec, 2019",0,131,tt5727208,movie,Uncut Gems,Uncut Gems,0.0,2019.0,\N,135,"Comedy,Crime,Drama",False,/snXY3M0WYpNGHFC4bYh6tv70H4B.jpg,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id':...",https://a24films.com/films/uncut-gems,473033.0,tt5727208,...,Uncut Gems,"Howard Ratner, a charismatic New Yor...",4.829,/7gOozJufKJ9WjcIs38KEs08Iq3D.jpg,"[{'id': 23243, 'logo_path': None, 'n...","[{'iso_3166_1': 'US', 'name': 'Unite...",2019-12-13,0.0,130.0,"[{'iso_639_1': 'en', 'name': 'Englis...",Released,This is how I win.,Uncut Gems,False,0.0,0.0,2019.0,Uncut Gems,AdamSandler KevinGarnett JoshSafdie ...,1966 1976 1984 \N \N \N \N \N \N \N,\N \N \N \N \N \N \N \N \N \N,NaN,NaN,"actor,producer,writer actor,director...",tt0142342 tt0120484 tt0120888 tt0185...
3,/worsethan/,6 Underground,2019,Watching this on Netflix felt wronge...,7,"17 Dec, 2019",0,57,tt8106534,movie,6 Underground,6 Underground,0.0,2019.0,\N,\N,"Action,Thriller",False,/u4UCu8GoM08G0B1is4gC0QYbnFm.jpg,NaN,150000000,"[{'id': 28, 'name': 'Action'}, {'id'...",NaN,509967.0,tt8106534,...,6 Underground,The plot revolves around six billion...,8.686,/lnWkyG3LLgbbrIEeyl5mK5VRFe4.jpg,"[{'id': 2481, 'logo_path': '/nVEP2IH...","[{'iso_3166_1': 'US', 'name': 'Unite...",2019-12-13,0.0,NaN,"[{'iso_639_1': 'en', 'name': 'Englis...",Post Production,They say no one can save the world. ...,6 Underground,False,0.0,0.0,2019.0,6 Underground,MichaelBay RyanReynolds IanBryce Mél...,1965 1976 \N 1983 \N \N \N 1983 1991 \N,\N \N \N \N \N \N \N \N \N \N,NaN,NaN,"producer,director,actor actor,produc...",tt1055369 tt2109248 tt0120591 tt0117...
4,

In [173]:
print("If every review entry took 5 seconds to process to generate people info, then total hours: {:.2f} \n\
Probably less than this but still a lot, better to use multiprocess or threading".format(300000*5/60/60/24))

If every review entry took 5 seconds to process to generate people info, then total hours: 17.36 
Probably less than this but still a lot, better to use multiprocess or threading


In [168]:
df_v3 = pd.merge(df_v2, title_ratings,
                 how='left', 
                 left_on=['tconst'], 
                 right_on = ['tconst'])

In [169]:
df_v3.head(3)

,user_name,movie_name,release_year,user_review,user_rating,review_date,rewatched,review_likes,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_orig,adult,backdrop_path,belongs_to_collection,budget,genres_tmdb,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,title_inferred,averageRating,numVotes
0,/worsethan/,Star Wars: The Rise of Skywalker,2019,"""I remember when I was with Special ...",10,"21 Dec, 2019",0,187,tt2527338,movie,Star Wars: The Rise of Skywalker,Star Wars: Episode IX - The Rise of ...,0.0,2019.0,\N,155,"Action,Adventure,Fantasy",False,/jOzrELAzFxtMx2I4uDGHOotdfsS.jpg,"{'id': 10, 'name': 'Star Wars Collec...",0,"[{'id': 28, 'name': 'Action'}, {'id'...",https://www.starwars.com/films/star-...,181812.0,tt2527338,en,Star Wars: The Rise of Skywalker,The next installment in the franchis...,34.725,/db32LaOibwEliAmSL2jjDF6oDdj.jpg,"[{'id': 1, 'logo_path': '/o86Dbpburj...","[{'iso_3166_1': 'JO', 'name': 'Jorda...",2019-12-18,0.0,155.0,"[{'iso_639_1': 'en', 'name': 'Englis...",Post Production,Every generation has a legend,Star Wars: The Rise of Skywalker,False,0.0,0.0,2019.0,Star Wars: The Rise of Skywalker,NaN,NaN
1,/worsethan/,The Report,2019,I realize that I prefer the Adam McK...,7,"16 Dec, 2019",0,48,tt8236336,movie,The Report,The Report,0.0,2019.0,\N,119,"Drama,Thriller",False,/yRNrOJyyeHJjgtJNLcqpo7rQyS6.jpg,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,524348.0,tt8236336,en,The Report,"The story of Daniel Jones, lead inve...",7.59,/sG5NI6TMNR9ftOdTIXmG0hrBYSY.jpg,"[{'id': 30288, 'logo_path': None, 'n...",[],2019-09-12,0.0,118.0,"[{'iso_639_1': 'en', 'name': 'Englis...",Released,Truth matters,The Report,False,7.3,6.0,2019.0,The Report,7.1,1247.0
2,/worsethan/,Uncut Gems,2019,All the greasy tri-state claustropho...,9,"18 Dec, 2019",0,131,tt5727208,movie,Uncut Gems,Uncut Gems,0.0,2019.0,\N,135,"Comedy,Crime,Drama",False,/snXY3M0WYpNGHFC4bYh6tv70H4B.jpg,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id':...",https://a24films.com/films/uncut-gems,473033.0,tt5727208,en,Uncut Gems,"Howard Ratner, a charismatic New Yor...",4.829,/7gOozJufKJ9WjcIs38KEs08Iq3D.jpg,"[{'id': 23243, 'logo_path': None, 'n...","[{'iso_3166_1': 'US', 'name': 'Unite...",2019-12-13,0.0,130.0,"[{'iso_639_1': 'en', 'name': 'Englis...",Released,This is how I win.,Uncut Gems,False,0.0,0.0,2019.0,Uncut Gems,8.0,913.0


## To Do ...

#1 ELIMINATE DUPLICATES BY A SELECTION FUNCTION BY POPULARITY (MORE NON-NULL DATA, ETC.)
#2 GENERATE THE PEOPLE FEATURES FOR THE WHOLE DATASET USING MULTI-THREADING...
#3 COMBINE SAME ENTITIES INTO JUST ONE COLUMNS (REMOVE JUNK)
